In [ ]:
from os import path
import cognee
import dspy
from cognee.modules.cognify.dataset import HotPotQA

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.prune.prune_system()

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url = "http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm = colbertv2_wiki17_abstracts)

dataset = HotPotQA(
    train_seed = 1,
    train_size = 10,
    eval_seed = 2023,
    dev_size = 0,
    test_size = 0,
    keep_details = True,
)

texts_to_add = []

for train_case in dataset.train:
    train_case_text = "\r\n".join(" ".join(context_sentences) for context_sentences in train_case.get("context")["sentences"])

    texts_to_add.append(train_case_text)

dataset_name = "train_dataset"
await cognee.add(texts_to_add, dataset_name)


In [ ]:
from os import path
import cognee

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

print(cognee.datasets.list_datasets())

train_dataset = cognee.datasets.query_data('train_dataset')
print(len(train_dataset))

In [ ]:
from os import path
import cognee

await cognee.prune.prune_system()

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

await cognee.cognify('train_dataset')

In [ ]:
import networkx as nx
import pandas as pd
import graphistry
from cognee.config import Config
from cognee.utils import render_graph
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType

config = Config()
config.load()

graphistry.register(
    api = 3,
    username = config.graphistry_username,
    password = config.graphistry_password
)

graph_client = await get_graph_client(GraphDBType.NETWORKX, "cognee_graph.pkl")
graph = graph_client.graph

await render_graph(graph)

# edges = nx.to_pandas_edgelist(graph)

# nodes_data = [{
#     "id": node_id,
#     "label": node["name"] if "name" in node else node_id,
# } for (node_id, node) in graph.nodes(data = True)]

# nodes = pd.DataFrame(nodes_data)

# plotter = graphistry.edges(edges, source = "source", destination = "target").nodes(nodes, "id")

# plotter.bind(edge_title = "relationship_name", edge_label = "relationship_name", point_title = "name", point_label = "name")
# url = plotter.plot(render = False, as_files = True)
# print(f"Graph is visualized at: {url}")


In [ ]:
from os import path
import cognee
from cognee.infrastructure.databases.graph.get_graph_client import get_graph_client, GraphDBType
from cognee.modules.search.vector.search_similarity import search_similarity

data_directory_path = path.abspath("../.data")
cognee.config.data_root_directory(data_directory_path)

cognee_directory_path = path.abspath("../.cognee_system")
cognee.config.system_root_directory(cognee_directory_path)

graph_client = await get_graph_client(GraphDBType.NETWORKX)
graph = graph_client.graph

results = await search_similarity("At My Window was released by which American singer-songwriter?", graph)

for result in results:
    print("At My Window" in result)
    print(result)